# Set-up of an AI-controlled self-driving Robo-Car

In this notebook, we will focus on detecting whether an image contains a stop sign. This is a binary classification task in the field of computer vision. Using TensorFlow, we will demonstrate how to implement and train a neural network to perform this classification task.

TensorFlow is an open-source framework for numerical computation and machine learning. We will cover the essential theoretical foundations of neural network-based classification and utilize TensorFlow to develop our model.

In [ ]:
# Import necessary libraries (This may take a while)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, regularizers
import matplotlib.pyplot as plt

from pathlib import Path
import datetime

print(tf.__version__)

## The Data

The datasets are provided separately. Later, you can use your own data and compare models trained with different data.
To work with our model, the images have to be 224x224 pixels in size.

The directory structure of every dataset should be as follows:

```
/path/to/dataset/
└───stop_signs/
│   │   image_with_stop_sign_1.jpg
│   │   ...
│   │   image_with_stop_sign_N.jpg
└───no_stop_signs/
    │   image_without_stop_sign_1.jpg
    │   ...
    │   image_without_stop_sign_N.jpg
```

We need both a training and a test dataset.

Tasks to accomplish:

- Load the training and test datasets from their respective directories.

In [ ]:
# TASK: Replace the paths with the path to your training and test data
train_dir = Path("data/train/")
test_dir = Path("data/test/")
# END TASK

# Counts the number of images
train_image_count = len(list(train_dir.glob('*/*.jpg')))
test_image_count = len(list(test_dir.glob('*/*.jpg')))
print("Train image count: ", train_image_count)
print("Test image count: ", test_image_count)

img_height = 224 
img_width = 224

dropout_rate = 0.3
dense_layer_size = 64
batch_size = 16

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
        test_dir,
        image_size=(img_height, img_width),
        batch_size=batch_size)

## Data Augmentation

To improve the variation of our training data, we can augment some images to have a different brightness or shift their pixels. If we do that, we have to make sure not to introduce unwanted side effects.

Tasks to accomplish:

- (Optional) Augment the training data.

In [ ]:
augmentation_probability = 0.5 # set to 0.0 to disable augmentation
brightness_vs_padding_probability = 0.7

def random_brightness(images, labels):
    """
    Apply random brightness to a batch of images.

    Parameters:
    -----------
    images: tf.Tensor
        A batch of images.
    labels: tf.Tensor
        A batch of labels.

    Returns:
    --------
    images_brightness: tf.Tensor
        A batch of images with random brightness.
    labels: tf.Tensor
        A batch of labels.
    """
    # TASK: Your code here
    # generate randomness
    seed = (tf.random.uniform([], 0, 10000, dtype=tf.int32), tf.random.uniform([], 0, 10000, dtype=tf.int32))
    # convert to float for brightness augmentation
    images_float = tf.cast(images, tf.float32) / 255.0
    images_brightness = tf.image.stateless_random_brightness(images_float, max_delta=0.4, seed=seed)
    # prevent black images (stop signs need to be recognizable)
    images_brightness = tf.clip_by_value(images_brightness, 0.2, 1.0)
    # convert back to uint8 for MobileNetV3
    images_brightness = tf.cast(images_brightness * 255.0, tf.uint8)
    return images_brightness, labels
    # END TASK

def random_shift_and_pad(images, labels):
    """
    Apply random shift and padding to a batch of images.

    Parameters:
    -----------
    images: tf.Tensor
        A batch of images.
    labels: tf.Tensor
        A batch of labels.

    Returns:
    --------
    padded_images: tf.Tensor
        A batch of images with random shift and padding.
    labels: tf.Tensor
        A batch of labels.
    """
    # TASK: Your code here
    # generate randomness
    shift_amount_x = tf.random.uniform([], -15, 15, dtype=tf.int32)
    shift_amount_y = tf.random.uniform([], -25, 25, dtype=tf.int32)
    pad_amount_x_before = tf.maximum(0, shift_amount_x)
    pad_amount_x_after = tf.maximum(0, -shift_amount_x)
    pad_amount_y_before = tf.maximum(0, shift_amount_y)
    pad_amount_y_after = tf.maximum(0, -shift_amount_y)
    # shift images
    images_roll = tf.roll(images, shift_amount_x, axis=2)
    images_roll = tf.roll(images_roll, shift_amount_y, axis=1)
    # crop parts that will be padded
    resized_images = tf.image.crop_to_bounding_box(
        images_roll, 
        tf.maximum(0, shift_amount_y), 
        tf.maximum(0, shift_amount_x), 
        img_height - tf.abs(shift_amount_y), 
        img_width - tf.abs(shift_amount_x)
    )
    # pad cropped parts, restores original image size
    padded_images = tf.pad(resized_images, 
        [[0, 0], [pad_amount_y_before, pad_amount_y_after], [pad_amount_x_before, pad_amount_x_after], [0, 0]],
        mode='CONSTANT', 
        constant_values=128
    )
    return padded_images, labels
    # END TASK

def augment(images, labels):
    """
    Apply random augmentation to a batch of images.

    Parameters:
    -----------
    images: tf.Tensor
        A batch of images.
    labels: tf.Tensor
        A batch of labels.

    Returns:
    --------
    augmented_images: tf.Tensor
        A batch of images with random augmentation.
    labels: tf.Tensor
        A batch of labels.
    """
    images = tf.cast(images, tf.uint8)

    if tf.random.uniform([]) > augmentation_probability:
        return images, labels
    if tf.random.uniform([]) < brightness_vs_padding_probability:
        return random_brightness(images, labels)
    else:
        return random_shift_and_pad(images, labels)
    
train_ds = train_ds.map(augment)

In [ ]:
# Visualize the first 9 training images

example_images, _ = next(iter(train_ds))

plt.figure(figsize=(10, 10))
for i in range(min(9, train_image_count)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(example_images[i].numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
# Visualize the first 9 test images

example_images, _ = next(iter(test_ds))

plt.figure(figsize=(10, 10))
for i in range(min(9, train_image_count)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(example_images[i].numpy().astype("uint8"))
    plt.axis("off")

## Building the Model

In this section, we delve into the construction of a neural network using the MobileNetV3 model, an architecture renowned for its efficiency on mobile and embedded vision applications. This part of the workshop will cover the integration and customization of MobileNetV3 for binary classification tasks.

We will load the MobileNetV3Small model with pre-trained ImageNet weights and freeze its base layers to leverage the learned features. The model is augmented with a few additional layers, including a dense layer, dropout, batch normalization, and a final output layer to perform binary classification.

Below is the code to load the MobileNetV3 model, selectively unfreeze a couple of its top layers, and build a sequential model using TensorFlow's Keras API. The model is then compiled and summarized to provide an overview of its architecture.

Tasks to accomplish:

- Add your own classification head to the MobileNetV3Small base model.

In [ ]:
# Load the MobileNetV3 model
base_model = keras.applications.MobileNetV3Small(
    input_shape=None,
    alpha=1.0,
    minimalistic=False,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    classes=None,
    pooling="avg",
    dropout_rate=dropout_rate,
    classifier_activation=None,
    include_preprocessing=True,
)

# Freeze the base model
trainable_layers = 2
base_model.trainable = False
for layer in base_model.layers[-trainable_layers:]:
    layer.trainable = True

# Add a classification head
model = models.Sequential([
    base_model,
    # TASK: Your code here
    layers.Dense(dense_layer_size, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(dropout_rate),
    layers.BatchNormalization(),
    # END TASK
    layers.Dense(1, activation='sigmoid') 
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
            loss='binary_crossentropy',
            metrics=[keras.metrics.BinaryAccuracy(name='accuracy')])

model.summary()

## Training the Model

To optimize the training process, we utilize Early Stopping and TensorBoard callbacks. Early Stopping monitors the validation loss and halts training if it doesn't improve for three consecutive epochs, restoring the best weights observed. TensorBoard logs training metrics, allowing detailed visualization and analysis. It is not required to use TensorBoard, but it can be useful if you want to improve the model. The model is trained with these callbacks to enhance performance and monitor progress.

In [ ]:
# early stopping
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',   # Beobachtete Metrik ist der Validierungsverlust
    patience=3,           # Anzahl der Epochen ohne Verbesserung bevor das Training gestoppt wird
    verbose=1,            # Zeigt eine Nachricht an, wenn das Training gestoppt wird
    restore_best_weights=True # Stellt die Gewichte der besten Epoche wieder her
)

# Tensorboard
log_dir = "logs/fit/" + "mobileNet_" + "Dataset_" + train_dir.name + "_Batch_size_" + str(batch_size) + "_dropout_rate_" + str(dropout_rate) + "_dense_layer_size_" + str(dense_layer_size) + "_" + str(datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
history = model.fit(
    train_ds, epochs=25, 
    validation_data=test_ds, 
    callbacks=[early_stopping_callback, tensorboard_callback]
)

In [ ]:
# Save the model
model.save("stop_sign_model.h5")

## Uploading to Raspberry Pi

We can upload the model to the Raspberry Pi using ssh. The first step is to connect the Raspberry Pi to the same network as us.

The easiest way to do this is to create a new OS image using the Raspberry Pi Imager tool. When creating the image, enable the SSH service and set the WiFi connection to a WiFi HotSpot of your Smartphone. Save the image onto the microSD card in the Raspberry Pi (you need an SD-Card reader for this). Enable your Smartphones WiFi Hotspot. Insert the microSD card back into the Raspberry Pi and power it. The Raspberry Pi is connected once you see it in your Smartphones HotSpot settings under "Connected devices" (The exact setting may vary depending on phone and operating system). There you should also find the IP address of the Raspberry Pi.

Connect your computer to the same WiFi HotSpot. Open VSCode and install the [Remote - SSH](https://marketplace.visualstudio.com/items?itemName=ms-vscode-remote.remote-ssh) extension. Use Ctrl+Shift+P to open the VSCode command prompt and select "Remote-SSH: Connect to Host...". Select "Add new Host" and type: ```ssh pi@<IP>```, where "\<IP\>" is the IP address of the Raspberry Pi. If VSCode asks for an operating system, select Linux. A new, remote VSCode window should now open, where you have access to the file system of the Raspberry Pi. Copy the model (stop_sign_model.h5) and the "roboCar.py" python script and paste it there.

## Test the Model on the RoboCar

Open a Terminal in the Remote VSCode window. Run the following commands to install the required libraries:

```bash
# TODO how did we install tensorflow, cv2, numpy?
```

Run ```python3 roboCar.py``` on the Raspberry Pi to start the car. Warning: It will start driving. Place a stop sign in front of its camera to see if the model works.

Important: Only stop the script (with Ctrl+C) if the car is already stopped! Otherwise it might not stop.